In [1]:
import pandas as pd
import requests
from pandas.io.json import json_normalize

In [ ]:
FMaug = 'https://www.commonsense.org/education/privacy/blog/digital-redlining-access-privacy'
dfAug = scrapeAPI(FMaug)

In [2]:
def scrapeAPI(url):
    n = 0
    payload = {'url':url}
    r = requests.get('https://hypothes.is/api/search',params=payload)
    df = pd.DataFrame(columns=['created', 'document.title', 'group', 'id', 'links.html',
       'links.incontext', 'links.json', 'permissions.admin',
       'permissions.delete', 'permissions.read', 'permissions.update',
       'references', 'tags', 'target', 'text', 'updated', 'uri', 'user'])
    total = r.json()['total']
    print('Number of Records Found: ',total)
    print('Scraped:')
    while n < total:
        print(n)
        df = df.append(json_normalize(r.json()['rows']))
        n = len(df)
        payload = {
            'url':'https://www.commonsense.org/education/privacy/blog/digital-redlining-access-privacy',
            'offset':n}
        r = requests.get('https://hypothes.is/api/search',params=payload)
    print(n)
    df['DateTime'] = pd.to_datetime(df['created'])
    df = df.set_index(pd.DatetimeIndex(df['DateTime']))

In [24]:
n = 0
payload = {'url':
           'https://www.commonsense.org/education/privacy/blog/digital-redlining-access-privacy'}
r = requests.get('https://hypothes.is/api/search',params=payload)
df = pd.DataFrame(columns=['created', 'document.title', 'group', 'id', 'links.html',
       'links.incontext', 'links.json', 'permissions.admin',
       'permissions.delete', 'permissions.read', 'permissions.update',
       'references', 'tags', 'target', 'text', 'updated', 'uri', 'user'])
total = r.json()['total']

In [25]:
while n < total:
    print(n)
    df = df.append(json_normalize(r.json()['rows']))
    n = len(df)
    payload = {
        'url':'https://www.commonsense.org/education/privacy/blog/digital-redlining-access-privacy',
        'offset':n}
    r = requests.get('https://hypothes.is/api/search',params=payload)
df['DateTime'] = pd.to_datetime(df['created'])
df = df.set_index(pd.DatetimeIndex(df['DateTime']))

0
20
40
60
80
100
120
140
160
180
